<a href="https://colab.research.google.com/github/Shuo-Zh/Shuo-Zh.github.io/blob/main/%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E6%9E%B6%E6%9E%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

kaiming初始化 == xviaer初始化，目的：不为了初次训练时挂掉

# pytorch 构建层和块

多层感知机

In [ ]:
# module 任何一个层和神经网络都是 module的子类
import torch
from torch import nn
from torch.nn import functional as F

# （单层神经网络）线性层 + relu + 线性层
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)) #10输出大小
# nn.Sequential 定义特殊module
X = torch.rand(2, 20) # 2 * 20的随机矩阵；2批量大小；20输入维度；
net(X)

tensor([[ 0.0391,  0.0956, -0.2009, -0.0964,  0.0611, -0.1332,  0.0550, -0.0204,
         -0.1253,  0.1215],
        [ 0.0348,  0.1023, -0.0064, -0.0372, -0.1212, -0.0553, -0.0016,  0.0562,
         -0.2344,  0.0408]], grad_fn=<AddmmBackward0>)

# 自定义块

任何一个层都是module子类

In [ ]:
class MLP(nn.Module):#定义类和参数
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20, 256) # 定义隐藏层
    self.out = nn.Linear(256, 10)
  def forward(self, X): #定义前向函数
    return self.out(F.relu(self.hidden(X))) #nnmodule里F函数，输入到隐藏层 -> 放到输出

实例化多层感知机，每次调用正向传播函数时调用这些层

In [ ]:
net = MLP()
net(X)

tensor([[ 0.1791, -0.1161, -0.0796, -0.1858,  0.0697,  0.1372,  0.1101,  0.0149,
          0.0276,  0.2784],
        [ 0.2524, -0.0433, -0.0547, -0.0918,  0.0016,  0.0500,  0.1110, -0.1143,
         -0.0981,  0.2241]], grad_fn=<AddmmBackward0>)

# 顺序块 - 方式1（Module子类）

In [ ]:
# 实现 NN- sequential功能
class MySequential(nn.Module):
    def __init__(self, *args): # *args == list of sequential argument
        super().__init__()
        # for idx, module in enumerate(args):
        #     # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
        #     # 变量_modules中。_module的类型是OrderedDict
        #     self._modules[str(idx)] = module
        for block in args:
          self._modules[block] = block

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

# 顺序块 - 方式2

In [ ]:
# 实现 NN- sequential功能
class MySequential2(nn.Module):
    def __init__(self, *args): # *args == list of sequential argument
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module


    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [ ]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1571,  0.1747,  0.1301, -0.1278, -0.2952, -0.1433,  0.0403,  0.0058,
          0.1042, -0.0231],
        [ 0.1809,  0.2699,  0.0915, -0.1841, -0.4316, -0.1792,  0.0655,  0.0908,
          0.1854,  0.0612]], grad_fn=<AddmmBackward0>)

# 正向传播函数中执行代码

In [ ]:
class FixedHiddenMLP(nn.Module):
  ## init和forward中可以做自定义计算
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # rand_weight不参与训练，不会计算梯度
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X) #先行回归
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1) #手写MM做矩阵乘法
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(-0.0774, grad_fn=<SumBackward0>)

## 反向计算自动求导

#混合搭配各种组合块的方法

In [ ]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.1820, grad_fn=<SumBackward0>)

#参数管理， 目标是找到使损失函数最小化的模型参数值。

In [ ]:
# 单隐藏层MLP
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.9516],
        [-0.9782]], grad_fn=<AddmmBackward0>)

## 参数访问

In [ ]:
#拿出权重
print(net[2].state_dict()) # 拿出最后的输出层

OrderedDict([('weight', tensor([[ 0.3428, -0.2510, -0.2270, -0.1447, -0.3183, -0.1937, -0.3358, -0.0941]])), ('bias', tensor([-0.2711]))])


In [ ]:
#目标参数
print(type(net[2].bias)) # Parameter，可以优化参数
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.2711], requires_grad=True)
tensor([-0.2711])


In [ ]:
net[2].weight.grad == None #weight 、grad==梯度

True

##一次性访问所有参数

In [ ]:
#relu没有参数访问不出来

# 一次性访问所有参数

In [ ]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


另一种网络参数访问方式

In [ ]:
net.state_dict()['2.bias'].data #根据名字获取参数

tensor([-0.2711])

# 从嵌套块收集参数

In [ ]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1()) #add model 可以传字符串
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0648],
        [0.0648]], grad_fn=<AddmmBackward0>)

In [ ]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


## 参数初始化

In [ ]:
def init_normal(m):# init 包含大量初始化函数
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) #normal下划线在后面，是替换函数
        nn.init.zeros_(m.bias)
net.apply(init_normal)# apply，所有net里layer，调用函数讲modole作为参数传入
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0031, -0.0109, -0.0017,  0.0010]), tensor(0.))

In [ ]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1) #weight，constant为1
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

## 应用不同的初始化方法

In [ ]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) #42是宇宙的答案

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.2220,  0.2091,  0.0549,  0.1629])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


##自定义初始化

In [ ]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5 #保留绝对值大于5的权重

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-7.6930, -0.0000,  6.3756,  0.0000],
        [-7.7345, -5.6781, -0.0000,  0.0000]], grad_fn=<SliceBackward0>)

##直接设置参数做替换

In [ ]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  7.3756,  1.0000])

##参数绑定 - 共享参数

In [ ]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), #第二个和第三个隐藏层share权重
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


# 自定义层

## 构造一个没有任何参数的的层

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [ ]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-4.6566e-09, grad_fn=<MeanBackward0>)

## 带参数的层

In [ ]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units): # in_units 输入大小与输出大小的矩阵，rand做初始化
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [ ]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.0834,  0.3246, -1.0586],
        [ 0.4155,  0.4027, -1.2503],
        [ 0.8679,  0.3451,  0.7627],
        [-0.3642,  0.8680,  0.7669],
        [-2.2921, -2.2194,  0.4480]], requires_grad=True)

In [ ]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[1.3961],
        [4.0682]])

## 带参数的图层

In [ ]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) #nn.parameter包起来
        self.bias = nn.Parameter(torch.randn(units,)) # randn做bias
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data #通过.data访问对应参数
        return F.relu(linear)

In [ ]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.9320, -0.4483,  3.2404],
        [-1.1338, -0.8829,  1.3985],
        [ 1.9408, -2.0297,  0.1764],
        [ 0.0722,  2.5382, -0.7770],
        [-2.2670, -0.8944,  0.4291]], requires_grad=True)

##自定义层执行正向计算

In [ ]:
linear(torch.rand(2, 5))

tensor([[0.0000, 1.6445, 1.4613],
        [0.0000, 0.3509, 3.6514]])

##自定义层构建模型

In [ ]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

#读写文件

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [ ]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

## 加载和保存模型参数

In [ ]:
# 不能存储模型定义，torchScript可以存模型定义，存权重即可
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [ ]:
torch.save(net.state_dict(), 'mlp.params')

## load回来模型

In [ ]:
clone = MLP()
clone.load_state_dict(torch.load("mlp.params"))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [ ]:
Y_clone = clone(X)
Y_clone = Y

# GPT 定义网络

In [ ]:
# 简单神经网络
import torch
import torch.nn as nn

# 定义一个简单的两层神经网络
class SimpleNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim) # 输入层到隐藏层的线性变换
        self.fc2 = nn.Linear(hidden_dim, output_dim) # 隐藏层到输出层的线性变换

    def forward(self, x):
        x = torch.relu(self.fc1(x)) # 对隐藏层使用ReLU激活函数
        x = self.fc2(x) # 输出层不需要激活函数
        return x

# 定义神经网络的输入、隐藏和输出层的维度
input_dim = 10
hidden_dim = 20
output_dim = 5

# 实例化神经网络
net = SimpleNet(input_dim, hidden_dim, output_dim)

# 定义一个输入张量
input_tensor = torch.randn(1, input_dim)

# 向前传播
output_tensor = net(input_tensor)

print(output_tensor)

tensor([[-0.0180, -0.4689, -0.0553, -0.0474,  0.2244]],
       grad_fn=<AddmmBackward0>)


In [ ]:
import torch
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 加载数据集
iris = load_iris()
X, y = iris.data, iris.target

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义一个简单的两层神经网络
class SimpleNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 定义神经网络的输入、隐藏和输出层的维度
input_dim = X.shape[1]
hidden_dim = 20
output_dim = len(set(y))

# 实例化神经网络
net = SimpleNet(input_dim, hidden_dim, output_dim)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# 训练神经网络
for epoch in range(100):
    # 将数据转换为张量
    inputs = torch.tensor(X_train, dtype=torch.float32)
    labels = torch.tensor(y_train, dtype=torch.long)

    # 将梯度清零
    optimizer.zero_grad()

    # 向前传播
    outputs = net(inputs)

    # 计算损失函数
    loss = criterion(outputs, labels)

    # 反向传播和优化
    loss.backward()
    optimizer.step()

# 在测试集上评估模型的准确性
inputs = torch.tensor(X_test, dtype=torch.float32)
labels = torch.tensor(y_test, dtype=torch.long)
outputs = net(inputs)
_, predicted = torch.max(outputs.data, 1)
accuracy = accuracy_score(predicted, labels)
print("Accuracy:", accuracy)

Accuracy: 0.7666666666666667


In [ ]:
import torch
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 加载数据集
iris = load_iris()
X, y = iris.data, iris.target

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义一个简单的两层神经网络
class SimpleNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 定义神经网络的输入、隐藏和输出层的维度
input_dim = X.shape[1]
hidden_dim = 50
output_dim = len(set(y))

# 实例化神经网络
net = SimpleNet(input_dim, hidden_dim, output_dim)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# 训练神经网络
for epoch in range(500):
    # 将数据转换为张量
    inputs = torch.tensor(X_train, dtype=torch.float32)
    labels = torch.tensor(y_train, dtype=torch.long)

    # 将梯度清零
    optimizer.zero_grad()

    # 向前传播
    outputs = net(inputs)

    # 计算损失函数
    loss = criterion(outputs, labels)

    # 反向传播和优化
    loss.backward()
    optimizer.step()

    # 每50个周期输出一次信息
    if epoch % 50 == 0:
        # 在测试集上评估模型的准确性
        inputs = torch.tensor(X_test, dtype=torch.float32)
        labels = torch.tensor(y_test, dtype=torch.long)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = accuracy_score(predicted, labels)
        print(f"Epoch [{epoch+1}/500], Loss: {loss.item():.4f}, Test Accuracy: {accuracy:.4f}")

# 在测试集上评估模型的准确性
inputs = torch.tensor(X_test, dtype=torch.float32)
labels = torch.tensor(y_test, dtype=torch.long)
outputs = net(inputs)
_, predicted = torch.max(outputs.data, 1)
accuracy = accuracy_score(predicted, labels)
print("Final Test Accuracy:", accuracy)

Epoch [1/500], Loss: 1.3591, Test Accuracy: 0.3667
Epoch [51/500], Loss: 0.7286, Test Accuracy: 0.8333
Epoch [101/500], Loss: 0.5287, Test Accuracy: 0.9000
Epoch [151/500], Loss: 0.4193, Test Accuracy: 0.9333
Epoch [201/500], Loss: 0.3449, Test Accuracy: 1.0000
Epoch [251/500], Loss: 0.2863, Test Accuracy: 1.0000
Epoch [301/500], Loss: 0.2383, Test Accuracy: 1.0000
Epoch [351/500], Loss: 0.1995, Test Accuracy: 1.0000
Epoch [401/500], Loss: 0.1692, Test Accuracy: 1.0000
Epoch [451/500], Loss: 0.1459, Test Accuracy: 1.0000
Final Test Accuracy: 1.0


隐藏层神经元数量增加到50，使用了Adam优化器，并将学习率设置为0.001。我们还将训练周期数增加到了500，并且在每50个周期输出一次信息，包括训练集上的损失函数和测试集上的准确性。最后，我们在测试集上评估了模型的准确性，并将结果输出。